In [1]:
from __init__ import *

In [2]:
directory_path = "{}/feature/".format(SETTING["directory_path"])

kraft.path.make(directory_path)

In [5]:
for url in ("https://ndownloader.figshare.com/files/22629137",):

    print(kraft.internet.download(url, directory_path))

/media/kwat/CarrotCake/garden/data/cancer_cell_line//feature/22629137 exists.
https://ndownloader.figshare.com/files/22629137 => /media/kwat/CarrotCake/garden/data/cancer_cell_line//feature/22629137...
/media/kwat/CarrotCake/garden/data/cancer_cell_line//feature/22629137


## Map cell line name

In [7]:
name_to_id = (
    pd.read_csv(
        "../input/cell-line-synon-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=1,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

id_to_rename = (
    pd.read_csv(
        "../input/cell-line-best-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

name_to_rename = {name.lower(): id_to_rename[id_] for name, id_ in name_to_id.items()}

len(name_to_rename)

10448

In [8]:
for name, rename in (
    pd.read_csv(
        "../input/ice000.cols.cell_line_identity.2777.csv",
        usecols=(2, 3),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .items()
):

    name_lower = name.lower()

    if name_lower in name_to_rename:

        rename_now = name_to_rename[name_lower]

        if rename_now != rename:

            print("{}: {} ==> {}".format(name, rename_now, rename))

    name_to_rename[name_lower] = rename

len(name_to_rename)

ACH-001163: PEDS005TPFAD ==> CCLFPEDS0001T
ACH-001164: PEDS015T ==> CCLFPEDS0003T
ACH-001206: TCC466 ==> TTC466
ACH-002124: H2373 ==> NCIH2373
ACH-002125: H2461 ==> NCIH2461
ACH-002126: H2591 ==> NCIH2591


11489

In [11]:
feature_x_cell_line = pd.read_csv(
    "{}22629137".format(directory_path), index_col=1
).T

In [13]:
for rename, names in feature_x_cell_line.iloc[:3, :].items():

    for name in names.dropna():

        name_lower = name.lower()

        if name_lower in name_to_rename:

            rename_now = name_to_rename[name_lower]

            if rename_now != rename:

                print("{}: {} ==> {}".format(name, rename_now, rename))

        name_to_rename[name_lower] = rename

len(name_to_rename)

NCI-H1581: NCIH1581 ==> NCIH2077
NCI-H2077: NCIH2077 ==> NCIH1581
D283: D283 ==> D283MED
ACH-000095: D341MED ==> D341
D341MED_CENTRAL_NERVOUS_SYSTEM: D341MED ==> D341
D341MED: D341MED ==> D341
SR: SR ==> SR786
ACH-000561: TTOESOPHAGUS ==> TDOTT
TT_OESOPHAGUS: TTOESOPHAGUS ==> TDOTT
ACH-000815: KMH2HAEMATOPOIETICANDLYMPHOIDTISSUE ==> KMH2
KMH2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE: KMH2HAEMATOPOIETICANDLYMPHOIDTISSUE ==> KMH2
CHL-1: CHL1 ==> HMCB
ACH-001007: A673STAG2KO16_ENGINEERED ==> A673STAG2KO16
ACH-001008: A673STAG2KO45_ENGINEERED ==> A673STAG2KO45
ACH-001009: A673STAG2NT14_ENGINEERED ==> A673STAG2NT14
ACH-001010: A673STAG2NT23_ENGINEERED ==> A673STAG2NT23
HMCB: HMCB ==> CHL1
PSN1: PSN1 ==> GRM
LC-1F: LC1F ==> LC1SQSF
ACH-001131: MS1SKIN ==> MSDASH1
MS1_SKIN: MS1SKIN ==> MSDASH1
MS-1: MS1LUNG ==> MSDASH1
R256_CENTRAL_NERVOUS_SYSTEM: R256_CENTRAL_NERVOUS_SYSTEM ==> R256
TCC466_BONE: TCC466 ==> TTC466
ACH-001225: UMRC6 ==> UMRC6NEO
UMRC6_KIDNEY: UMRC6 ==> UMRC6NEO
UPCISCC090_UPPER_AERO

11824

In [14]:
for rename in tuple(name_to_rename.values()):

    name_to_rename[rename.lower()] = rename

len(name_to_rename)

12805

In [15]:
name_to_rename = pd.Series(name_to_rename).sort_values()

name_to_rename.index.name = "Name"

name_to_rename.name = "Rename"

name_to_rename.to_csv(
    "{}cell_line_name_rename.tsv".format(directory_path), sep="\t", header=True
)

name_to_rename

Name
ach-001435       1156QE8
1156qe8          1156QE8
1205lu            1205LU
ach-002633        1205LU
127399            127399
                 ...    
idcell:933    idCell:933
cddb:867      idCell:934
idcell:934    idCell:934
idcell:935    idCell:935
cddb:868      idCell:935
Name: Rename, Length: 12805, dtype: object

## Make feature_x_cell_line

### All

In [16]:
feature_x_cell_line.loc["Is Fibroblast", :] = tuple(
    int(bool(re.match(".*FIBROBLAST$", name)))
    for name in feature_x_cell_line.loc["CCLE_Name", :].to_numpy()
)

feature_x_cell_line.index.name = "Feature"

feature_x_cell_line.columns.name = "Cell Line"

kraft.table.error_axes(feature_x_cell_line)

feature_x_cell_line.to_csv(
    "{}feature_x_cell_line.tsv".format(directory_path), sep="\t"
)

feature_x_cell_line

AttributeError: module 'kraft' has no attribute 'table'

### Continuous

In [17]:
feature_x_cell_line_x_continuous = pd.concat((feature_x_cell_line.loc[["age"], :],))

feature_x_cell_line_x_continuous.to_csv(
    "{}feature_x_cell_line_x_continuous.tsv".format(directory_path), sep="\t"
)

feature_x_cell_line_x_continuous

Cell Line,NIHOVCAR3,HL60,CACO2,HEL,HEL9217,MONOMAC6,LS513,A101D,C2BBE1,NCIH2077,...,CCLFUPGI0005T,HT144SKINFV1,HT144SKINFV3,HT144SKINFV2,RVH421SKINFV1,HAP1,WM3211,WM4235,M040416,M140325
Feature,,,,,,,,,,,,,,,,,,,,,
age,60,35,NaN,30,30,64,63,56,72,NaN,...,NaN,29,29,29,NaN,40,34,NaN,54,64


### Binary

In [18]:
dataframes = [feature_x_cell_line.loc[["Is Fibroblast"], :]]

for label in (
    "sex",
    "sample_collection_site",
    "primary_or_metastasis",
    "primary_disease",
    "Subtype",
    "lineage",
    "lineage_subtype",
    "lineage_sub_subtype",
    "lineage_molecular_subtype",
):

    dataframe = kraft.table.binarize(feature_x_cell_line.loc[label, :])

    dataframe.index = (
        "({}) {}".format(label, label_) for label_ in dataframe.index.to_numpy()
    )

    dataframes.append(dataframe)

feature_x_cell_line_x_01 = pd.concat(dataframes)

feature_x_cell_line_x_01.index.name = "Feature"

feature_x_cell_line_x_01.to_csv(
    "{}feature_x_cell_line_x_01.tsv".format(directory_path), sep="\t"
)

feature_x_cell_line_x_01

AttributeError: module 'kraft' has no attribute 'table'